# **Previsão de preço de carros.**
<!--
<img src="https://miro.medium.com/v2/resize:fit:988/0*tA5OjppLK627FfFo" alt="image" width="250" height="auto">
-->
Nesse projeto eu vou construir um modelo de **Machine Learning** que faça a previsão  do preço de venda de um veículo.

Irei utilizar a Metodologia **CRISP-DM** para me basear nos passos a serem seguinos no projeto.

1. **Compreensão do Negócio (Business Understanding/Problem)** 

1. **Compreensão dos Dados (Data Understanding)** 
1. **Preparação dos Dados (Data Preparation)** 
1. **Modelagem (Modeling)** 
1. **Avaliação (Evaluation)**
1. **Implantação (Deployment)** 

# **1. Problema de Negóscio**

*   Uma pessoa precisa revender seu veículo porém ela **não sabe** qual valor deve cobrar no mesmo.
*   1.1 Qual é o contexto?
    *   ...
*   1.2 Quais são os objetivos do projeto?
    *   Identificar as características que influenciam os preços de venda dos veículos.
    *   Criar um modelo capaz de prever o valor de venda de um novo veículo.
    *   ...
*   1.3 Quais são os benefícios?
    *   Vender o veículo com o valor *ótimo* de acordo com **suas características**.
    *   **Reduzir as perdas de dinheiro** na revenda do veículo.
    *   **Enconomizar esforços de energia/tempo** em tentar adivinhar o valor ideal.

Como resultado, o **problema de negócio** será resolvido.

### **Importando as bibliotecas necessárias.**

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib as mpl
import seaborn as sns

# Display options.
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 100)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.width', 1000)

# **2. Compreendendo os dados**
*   Os dados foram obtidos através de web scraping do site  [Olx](https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?) e **são de propiedade de olx.com**
*   O repositório do projeto de web scraping completo se encontra [aqui](https://github.com/PatrickLeal/olx-veiculos-web-scraping).
*   Os dados contém informações sobre os veículos anunciados no site. 

In [10]:
silver_data_caminho = '../data_lake/silver/20240913_132209_dados_veiculos_silver.csv'
df_carros = pd.read_csv(silver_data_caminho, parse_dates=['DATA_PUBLICACAO'])

In [29]:
df_carros.head(3)

,TITULO,PRECO_BRL,DESCRICAO,OPCIONAIS,CEP,IMAGEM,PERFIL_CARRO,DATA_PUBLICACAO,TEMPO_PUBLICACAO,COD_PUBLICACAO,TIPO_ANUNCIO,ANO,MARCA,CAMBIO,POSSUI_KIT_GNV,PORTAS,POTENCIA_DO_MOTOR,TIPO_DE_VEICULO,FINAL_DE_PLACA,COR,QUILOMETRAGEM,COMBUSTIVEL,TIPO_DE_DIREÇÃO,MODELO,CATEGORIA
0,Fiat Mobi 2023 1.0 evo flex like. manual,53590.00,FIAT MOBI 1.0 EVO FLEX LIKE. MANUAL - 2023 / 2...,"['Ar condicionado', 'Trava elétrica']",71200020,https://img.olx.com.br/images/46/4634813239611...,https://df.olx.com.br/distrito-federal-e-regia...,2024-09-09,19:21,1335272425,profissional,2023,FIAT,Manual,Não,4 portas,1.0,Hatch,1.00,Branco,45021,Flex,Hidráulica,FIAT MOBI LIKE 1.0 FIRE FLEX 5P.,"Carros, vans e utilitários"
1,Omega CD 3.0,39900.00,Carro conservado tem potencial Completo Teto s...,"['Alarme', 'Air bag', 'Ar condicionado', 'Banc...",8030070,https://img.olx.com.br/images/98/9874565435056...,https://sp.olx.com.br/sao-paulo-e-regiao/autos...,2024-09-09,19:21,1260360322,NaN,1994,CHEVROLET,Manual,Não,4 portas,4.0 ou mais,Sedã,3.00,Azul,156000,Gasolina,Hidráulica,CHEVROLET OMEGA CD 4.1 / 3.0,"Carros, vans e utilitários"
2,VW Fox Peper Troco Por maior valor,52000.00,VW Fox Peper 1.6 o mais top da categoria impec...,"['Alarme', 'Air bag', 'Ar condicionado', 'Câme...",13203280,https://img.olx.com.br/images/19/1944761991382...,https://sp.olx.com.br/sao-paulo-e-regiao/autos...,2024-09-09,19:21,1333110058,NaN,2017,VOLKSWAGEN,Automatizado,Não,4 portas,1.6,Hatch,7.00,Prata,88700,Flex,Elétrica,VOLKSWAGEN FOX PEPPER I MOTION 1.6 FLEX 16V 5P,"Carros, vans e utilitários"


### **Dicionário de dados**

|VARIÁVEL| DESCRIÇÃO | TIPO |
|--------|-----------|------|
|**TITULO:**| Título do anúncio| string |
|**PRECO_BRL:**| Preço em Reais R$| int |
|**DESCRICAO:**| Descrição do anúncio deixada pelo anunciante| string |
|**OPCIONAIS:**| São *features* dos veículos informadas pelo anunciante| string |
|**CEP:**| Localização veículo deixada pelo anunciante | int |
|**IMAGEM:**| Link da foto de perfil do anúncio | string |
|**PERFIL_CARRO:**| Link para o anúncio original | string |
|**DATA_PUBLICACAO:**| Date em que o anúncio foi criado| date |
|**TEMPO_PUBLICACAO:**| Hora em que o anúncio foi criado| time |
|**COD_PUBLICACAO:**| Código do anúncio| int |
|**TIPO_ANUNCIO:**| Tipo de anúncio| string|
|**ANO:**| Informa o Ano doveículo| int |
|**MARCA:**| Nome da marca do veículo| string |
|**CAMBIO:**| Tipo de câmbio do veículo | string |
|**POSSUI_KIT_GNV:**| Se o veículo possui kit GNV| bool |
|**PORTAS:**| Quantidade de portas que o veículo possui| int |
|**POTENCIA_DO_MOTOR:**| Informa a potência do motor | float |
|**TIPO_DE_VEICULO:**| Informa o tipo de veículo |string |
|**FINAL_DE_PLACA:**| Informa o final da placa| int |
|**COR:**| Informa a cor do veículo| string |
|**QUILOMETRAGEM:**| Informa a quilometragem do veículo| int |
|**COMBUSTIVEL:**| Informa o o tipo de combustível usado pelo veículo| string|
|**TIPO_DE_DIREÇÃO:**| Informa o tipo de direção do veículo| string |
|**MODELO:**| Informa o modelo do veículo|string |
|**CATEGORIA:**| Informa e qual categoria o veículo foi anunciado |string |

### **Informações gerais dos dados:**

In [30]:
df_carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13469 entries, 0 to 13468
Data columns (total 25 columns):
 #   Column             Dtype         
---  ------             -----         
 0   TITULO             object        
 1   PRECO_BRL          float64       
 2   DESCRICAO          object        
 3   OPCIONAIS          object        
 4   CEP                int64         
 5   IMAGEM             object        
 6   PERFIL_CARRO       object        
 7   DATA_PUBLICACAO    datetime64[ns]
 8   TEMPO_PUBLICACAO   object        
 9   COD_PUBLICACAO     int64         
 10  TIPO_ANUNCIO       object        
 11  ANO                object        
 12  MARCA              object        
 13  CAMBIO             object        
 14  POSSUI_KIT_GNV     object        
 15  PORTAS             object        
 16  POTENCIA_DO_MOTOR  object        
 17  TIPO_DE_VEICULO    object        
 18  FINAL_DE_PLACA     float64       
 19  COR                object        
 20  QUILOMETRAGEM      int64    

In [31]:
print(f"O dataset possui {df_carros.shape[0]} linhas e {df_carros.shape[1]} colunas.")

O dataset possui 13469 linhas e 25 colunas.


**Visualisando algumas estatísticas:**

In [51]:
df_carros.describe().T

,count,mean,min,25%,50%,75%,max,std
PRECO_BRL,13354.00,89255.13,0.00,53290.00,72290.00,106990.00,1190000.00,67758.23
CEP,13469.00,40611217.01,1013010.00,11726000.00,32215000.00,71200020.00,99500000.00,30371801.12
DATA_PUBLICACAO,13469,2024-09-10 18:50:42.185759744,2024-07-23 00:00:00,2024-09-09 00:00:00,2024-09-10 00:00:00,2024-09-13 00:00:00,2024-09-13 00:00:00,NaN
COD_PUBLICACAO,13469.00,1329814988.06,614262154.00,1329370471.00,1334425267.00,1336417075.00,1337433403.00,17415190.43
FINAL_DE_PLACA,11341.00,4.40,0.00,2.00,4.00,7.00,9.00,2.86
QUILOMETRAGEM,13469.00,69470.42,0.00,42844.00,52593.00,90000.00,850000.00,55973.79


**Análise:**
-   o dataset possui anúncios com o **preço** de venda **igual a 0**, o que não faz sentido nesse contexto
-   alguns dados estão com **tipos errados** e precisam ser tradados

**Verificando valores nulos e duplicados:**

In [39]:
print("Porcentagem de valores nulos.")
(df_carros.isnull().sum() / df_carros.shape[0]) * 100

Porcentagem de valores nulos.


TITULO               0.00
PRECO_BRL            0.85
DESCRICAO            0.00
OPCIONAIS            0.00
CEP                  0.00
IMAGEM               0.00
PERFIL_CARRO         0.00
DATA_PUBLICACAO      0.00
TEMPO_PUBLICACAO     0.00
COD_PUBLICACAO       0.00
TIPO_ANUNCIO        23.80
ANO                  0.00
MARCA                0.00
CAMBIO               0.38
POSSUI_KIT_GNV       0.00
PORTAS               1.82
POTENCIA_DO_MOTOR    5.38
TIPO_DE_VEICULO     13.88
FINAL_DE_PLACA      15.80
COR                  1.83
QUILOMETRAGEM        0.00
COMBUSTIVEL          0.51
TIPO_DE_DIREÇÃO      7.46
MODELO               0.00
CATEGORIA            0.00
dtype: float64

In [46]:
print(f"Quantidade de valores duplicados: {df_carros.duplicated().sum()}")

Quantidade de valores duplicados: 2080


**Ações a serem tomada:**
*   remover duplicatas
*   remover colunas que tenham mais de 15% de valores nulos
*   alterar os tipos de dados das colunas:
    1. ANO
    1. POSSUI_KIT_GNV
    1. PORTAS
    1. POTENCIA_DO_MOTOR
    1. FINAL_DE_PLACA